
# Plant Disease Recognition - CNN Model

## Objectif
Classification de maladies des plantes en 3 catégories : Healthy, Powdery, Rust

## Dataset
Source : Kaggle - Plant Disease Recognition Dataset




In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rashikrahmanpritom/plant-disease-recognition-dataset")

print("Path to dataset files:", path)

C:\Users\lenovo\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\lenovo\.cache\kagglehub\datasets\rashikrahmanpritom\plant-disease-recognition-dataset\versions\1


## Configuration du versioning avec DVC

In [ ]:
# Initialiser DVC pour tracker le dataset
!pip install dvc dvc-gdrive -q

# Monter Google Drive pour stockage DVC
from google.colab import drive
drive.mount('/content/drive')

# Initialiser DVC
!dvc init -f

# Configurer le stockage distant sur Google Drive
!mkdir -p /content/drive/MyDrive/plant_disease_dvc_storage
!dvc remote add -d storage /content/drive/MyDrive/plant_disease_dvc_storage -f

print(" DVC configuré")

In [ ]:
import os

def total_files(folder_path):
    # Compte uniquement les fichiers
    return len([
        f for f in os.listdir(folder_path)
        if os.path.isfile(os.path.join(folder_path, f))
    ])

# Base path du dataset (COLAB)
base_path = path # Corrected path to use the downloaded dataset location

# Train
train_files_healthy = os.path.join(base_path, "Train", "Train", "Healthy")
train_files_powdery = os.path.join(base_path, "Train", "Train", "Powdery")
train_files_rust = os.path.join(base_path, "Train", "Train", "Rust")

# Test
test_files_healthy = os.path.join(base_path, "Test", "Test", "Healthy")
test_files_powdery = os.path.join(base_path, "Test", "Test", "Powdery")
test_files_rust = os.path.join(base_path, "Test", "Test", "Rust")

# Validation
valid_files_healthy = os.path.join(base_path, "Validation", "Validation", "Healthy")
valid_files_powdery = os.path.join(base_path, "Validation", "Validation", "Powdery")
valid_files_rust = os.path.join(base_path, "Validation", "Validation", "Rust")

# Affichage
print("=== TRAIN SET ===")
print("Healthy:", total_files(train_files_healthy))
print("Powdery:", total_files(train_files_powdery))
print("Rust:", total_files(train_files_rust))

print("=== TEST SET ===")
print("Healthy:", total_files(test_files_healthy))
print("Powdery:", total_files(test_files_powdery))
print("Rust:", total_files(test_files_rust))

print("=== VALIDATION SET ===")
print("Healthy:", total_files(valid_files_healthy))
print("Powdery:", total_files(valid_files_powdery))
print("Rust:", total_files(valid_files_rust))


## 2. Visualisation d'exemples


In [ ]:
from PIL import Image
import IPython.display as display
import os

# Chemin vers l'image (CORRIGÉ)
image_path = "/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Train/Train/Healthy/800edef467d27c15.jpg"

# Vérifier si le fichier existe
if os.path.exists(image_path):
    with open(image_path, 'rb') as f:
        display.display(display.Image(data=f.read(), width=500))
else:
    print("Fichier introuvable :", image_path)


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

image_path = "/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Train/Train/Powdery/8a2d598f2ec436e6.jpg"

if os.path.isfile(image_path):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
else:
    print("Fichier introuvable :", image_path)


In [ ]:
from PIL import Image
import IPython.display as display
import os

# Chemin vers l'image (Colab / Linux)
image_path = "/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Train/Train/Rust/8a954b82bf81f2bc.jpg"

# Vérifier si le fichier existe
if os.path.isfile(image_path):
    with open(image_path, 'rb') as f:
        display.display(display.Image(data=f.read(), width=500))
else:
    print("Fichier introuvable :", image_path)


## 3. Préparation des données
Data augmentation et création des générateurs

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Générateur pour les images d'entraînement avec augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,        # Normalisation
    shear_range=0.2,       # Cisaillement aléatoire
    zoom_range=0.2,        # Zoom aléatoire
    horizontal_flip=True   # Flip horizontal
)

# Générateur pour les images de test (ou validation)
test_datagen = ImageDataGenerator(
    rescale=1./255
)

# Chemins du dataset (Colab / Linux)
train_dir = "/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Train/Train"
test_dir  = "/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Test/Test"

# Chargement des images d'entraînement
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(225, 225),
    batch_size=32,
    class_mode='categorical'
)

# Chargement des images de test
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(225, 225),
    batch_size=32,
    class_mode='categorical',
    shuffle=False   # IMPORTANT pour matrice de confusion
)


## 4. Construction du modèle CNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# 1ère couche convolutionnelle
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(225, 225, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 2ème couche convolutionnelle
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Couches fully connected
model.add(Flatten())
model.add(Dense(64, activation='relu'))

# Couche de sortie (3 classes)
model.add(Dense(3, activation='softmax'))


## 5. Compilation du modèle

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## 6. Visualisation de l'architecture

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

## 7. Entraînement du modèle

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    train_set,
    epochs=20,
    validation_data=test_set,
    callbacks=[early_stop]
)


## Sauvegarde et versioning du modèle

In [ ]:
# Sauvegarder le modèle entraîné
model.save('plant_disease_model.h5')
print("✅ Modèle sauvegardé : plant_disease_model.h5")

# Sauvegarder l'historique d'entraînement
import pickle
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("✅ Historique sauvegardé : training_history.pkl")

# Tracker avec DVC
!dvc add plant_disease_model.h5
!dvc add training_history.pkl

# Pousser vers le stockage distant (Google Drive)
!dvc push

print("✅ Modèle et historique trackés avec DVC")

## 8. Analyse des performances

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import seaborn as sns
sns.set_theme()
sns.set_context("poster")

figure(figsize=(10, 8), dpi=80)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
figure(figsize=(10, 8), dpi=80)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## 9. Évaluation finale sur le test set

In [ ]:
test_generator = test_datagen.flow_from_directory('/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Test/Test',
                                                        target_size=(225, 225),
                                                        batch_size=32,
                                                        class_mode='categorical')

model.evaluate(test_generator)

## 10. Prédiction sur une image individuelle

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

def preprocess_image(image_path, target_size=(225, 225)):
    # load image
    img = load_img(image_path, target_size=target_size)

    # convert to array and preprocess
    x = img_to_array(img)
    x = x.astype('float32') / 255.
    x = np.expand_dims(x, axis=0)

    return x

x = preprocess_image('/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Test/Test/Rust/82f49a4a7b9585f1.jpg')


In [ ]:
predictions = model.predict(x)
predictions[0]

In [ ]:
labels = train_set.class_indices
labels = {v: k for k, v in labels.items()}
labels

In [ ]:
predictions = model.predict(x)
predicted_label = labels[np.argmax(predictions)]
print(predicted_label)

In [ ]:
# Afficher l'image avec la prédiction
img_path = '/root/.cache/kagglehub/datasets/rashikrahmanpritom/plant-disease-recognition-dataset/versions/1/Test/Test/Rust/82f49a4a7b9585f1.jpg'
if os.path.exists(img_path):
    with open(img_path, 'rb') as f:
        display.display(display.Image(data=f.read(), width=400))
    print(f"Prédiction : {predicted_label}")
    print(f"Confiance : {np.max(predictions[0]):.2%}")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Prédictions sur tout le test set
y_pred = model.predict(test_set)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_set.classes

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels.values(),
            yticklabels=labels.values())
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Rapport de classification
print(classification_report(y_true, y_pred_classes, target_names=labels.values()))

In [ ]:
# Test rapide
try:
    print("Type du modèle:", type(model))
    print("✅ Le modèle existe en mémoire!")
    print(f"Classes de sortie: {model.output_shape}")
except NameError:
    print("❌ Le modèle n'existe pas en mémoire")
    print("⚠️ Vous devez exécuter les cellules d'entraînement d'abord")